# Fun with language modelling

Если вы пропустили лекцию, то посмотрите слайды к ней — они где-то есть. Также полезно почитать:

* [Unreasonable effectiveness of RNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) (Andrej Karpathy)
* [Официальный пример от PyTorch](https://github.com/pytorch/examples/tree/master/word_language_model)

Рекомендуется заранее всё прочитать, чтобы понять, что от вас хотят. При желании, можете переписать всё так, как подсказывает ваше сердце.

---

In [47]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

import numpy as np
from tqdm.autonotebook import tqdm
import re

import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
SEED = 42
np.random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [50]:
from google.colab import drive
import os

drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


## Препроцессинг (3 балла)

Возьмите какие-нибудь сырые данные. Википедия, «Гарри Поттер», «Игра Престолов», тексты Монеточки, твиты Тинькова — что угодно.

Для простоты будем делать char-level модель. Выкиньте из текстов все ненужные символы (можете оставить только алфавит и, пунктуацию). Сопоставьте всем различным символам свой номер. Удобно это хранить просто в питоновском словаре (`char2idx`). Для генерации вам потребуется ещё и обратный словарь (`idx2char`). Вы что-то такое должны были писать на вступительной — можете просто переиспользовать код оттуда.

Заранее зарезервируйте айдишники под служебные символы: `<START>`, `<END>`, `<PAD>`, `<UNK>`.

Клёво будет написать отдельный класс, который делает токенизацию и детокенизацию.

In [51]:
class Vocab:
    def __init__(self, data, token_size=100):
        
        self.data = data
        self.token_size = token_size
        
        self.char2idx = {
            '<START>': 0, 
            '<END>': 1, 
            '<PAD>': 2, 
            '<UNK>': 3
        }
        self.idx2char = {
            0: '<START>', 
            1: '<END>', 
            2: '<PAD>', 
            3: '<UNK>'
        }
        self.count = 4
        
        for symbol in data:
            if symbol not in self.char2idx:
                self.char2idx[symbol] = self.count
                self.idx2char[self.count] = symbol
                self.count += 1

    def tokenize_data(self):
        # выполните какой-то базовый препроцессинг
        # например, оставьте только алфавит и пунктуацию
        self.data = re.sub(r"[^A-zА-я.!?,–:]+", " ", self.data)
        self.data = re.sub("\s+", " ", self.data)
        return [0] + [self.char2idx.get(char, 3) for char in self.data] + [1]
    
    def tokenize(self, sequence):
        return [self.char2idx[char] for char in sequence]

    def detokenize(self, sequence):
        return ''.join([self.idx2char[idx] for idx in sequence])
    
    def add_pad(self, tokenized):
        return tokenized + [self.char2idx['<PAD>']]*max(0, self.token_size-len(tokenized))

    def __len__(self):
        return len(self.char2idx)

In [52]:
class TextDataset():
    
    def __init__(self, data_path, fix_size=100, min_size=40):
        self.fix_size = fix_size
        # загрузите данные
        data = open(data_path, 'r').read()

        # обучите вокаб
        self.vocab = Vocab(data, self.fix_size)

        # разделите данные на отдельные сэмплы для обучения (просто список из сырых строк)
        # Вообще можно было бы сделать как предлагается в ноутбуке, но тогда костыльно приходится впихнуть
        # служебные символы и в нарезаемые куски фиксированной длины могут попасть ненужные символы, после удаления
        # которых длина подаваемой строки уменьшится. Выход: токенизировать сразу исходный текст с его предобработкой.
        data = self.vocab.tokenize_data()
        slice_counts = len(data) // self.fix_size + 1
        self.data = [data[i*self.fix_size:(i+1)*self.fix_size] for i in range(slice_counts)]
        if min_size <= len(self.data[-1]) <= fix_size:
            self.data[-1] = self.vocab.add_pad(self.data[-1][:-1])
            self.data[-1][-1] =  self.vocab.char2idx['<END>']
        else:
            self.data.pop(-1)
            self.data[-1][-1] =  self.vocab.char2idx['<END>']

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        sample = self.vocab.add_pad(sample) # паддинг до fix_size
        sample = torch.LongTensor(sample) # сконвертируйте в LongTensor
        target = sample[1:] # нужно предсказать эту же последовательность со сдвигом 1
        target = torch.cat([sample[1:], torch.tensor([self.vocab.char2idx['<PAD>']])])
        return sample, target

    def get_tokenized(self):
        return self.data
    
    def get_text(self, idx):
        return self.vocab.detokenize(self.data[idx])

Если у вас какой-то большой массив текста (скажем, статьи Википедии), вы можете просто нарезать из него кусочки фиксированной длины и так их подавать в модель.

Если же вы хотите приключений, то можно разбить этот текст на предложения (`nltk.sent_tokenize`), и тогда все примеры будут разной длины. По соображениям производительности, вы не хотите использовать самые длинные и самые короткие сэмплы, поэтому имеет смысл обрезать их по длине.

Будем резать текст по token_len символов. За основу возьмем роман М. Булгакова "Мастер и Маргарита".

In [53]:
token_len = 150

In [54]:
dataset = TextDataset('/content/gdrive/MyDrive/Colab Notebooks/Tinkoff DL/Занятие 8/Bulgakov.txt', token_len)

In [55]:
for i in range(10):
    print(dataset.get_text(i))
    print('_____________________'*5)

# Посмотрим ещё и на последний элемент
print(dataset.get_text(-1))

<START>Михаил Булгаков Мастер и Маргарита ЧАСТЬ ПЕРВАЯ ...Так кто ж ты, наконец? – Я – часть той силы, что вечно хочет зла и вечно совершает благо. Гете. Фа
_________________________________________________________________________________________________________
уст Глава Никогда не разговаривайте с неизвестными Однажды весною, в час небывало жаркого заката, в Москве, на Патриарших прудах, появились два гражда
_________________________________________________________________________________________________________
нина. Первый из них, одетый в летнюю серенькую пару, был маленького роста, упитан, лыс, свою приличную шляпу пирожком нес в руке, а на хорошо выбритом
_________________________________________________________________________________________________________
 лице его помещались сверхъестественных размеров очки в черной роговой оправе. Второй – плечистый, рыжеватый, вихрастый молодой человек в заломленной 
________________________________________________________________________

Разобьём на обучение и валидацию:

In [56]:
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

## Модель (3 балла)

Примерно такое должно зайти:

* Эмбеддинг
* LSTM / GRU
* Линейный слой
* Softmax

In [57]:
class LM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout, tie_weights):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.encoder = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)
        self.decoder = nn.Linear(hidden_dim, vocab_size)
        self.drop = nn.Dropout(dropout)
        
        if tie_weights:
            # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
            # https://arxiv.org/abs/1608.05859
            assert hidden_dim == embedding_dim
            self.decoder.weight = self.encoder.weight

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.reshape(output.size(0)*output.size(1), output.size(2)))
        return decoded.reshape(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, batch_size):
        # начальный хидден должен быть нулевой
        # (либо хоть какой-то константный для всего обучения)
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim)

## Обучение

In [59]:
epochs = 10
lr = 1e-3 
batch_size = 64

# device = torch.device('cpu')

model = LM(
    vocab_size = len(dataset.vocab), # = 134
    embedding_dim = 128,
    hidden_dim = 128,
    num_layers = 10, # добавляя dropout надо количество слоев делать больше 1
    dropout = 0.1,
    tie_weights= True
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test = DataLoader(test_set, batch_size=batch_size)

In [60]:
train_loss = []
test_loss = []

for epoch in tqdm(range(epochs)):
    train_running_loss = 0.0
    test_running_loss = 0.0
    for x, y in tqdm(train, leave=False, total=len(train)):
        model.train()
        
        model.zero_grad()
        
        # 0. Распакуйте данные на нужное устройство
        x = x.to(device)
        y = y.to(device)

        # 1. Инициилизируйте hidden
        hidden = model.init_hidden(len(x)).to(device)
        
        # 2. Прогоните данные через модель, получите предсказания на каждом токене
        output, hidden = model(x, hidden)
        
        # 3. Посчитайте лосс (maxlen независимых классификаций) и сделайте backward()
        # Преобразуем полученные вектора, чтобы можно было передать их в criterion
        y = y.reshape(output.shape[0] * output.shape[1])
        output = output.reshape(-1, len(dataset.vocab))
        loss = criterion(output, y)
        loss.backward()
        
        # 4. Клипните градиенты -- у RNN-ок с этим часто бывают проблемы
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.2)
        
        # 5. Залоггируйте лосс куда-нибудь
        train_running_loss += loss.item()
        train_loss.append(loss.item())

        optimizer.step()
    
    for x, y in tqdm(test, leave=False, total=len(test)):
        model.eval()
        with torch.no_grad():
            # здесь нужно сделать то же самое, только без backward
            x = x.to(device)
            y = y.to(device)

            hidden = model.init_hidden(len(x)).to(device)
            
            output, hidden = model(x, hidden)

            y = y.reshape(output.shape[0] * output.shape[1])
            output = output.reshape(-1, len(dataset.vocab))
            loss = criterion(output, y)

        test_running_loss += loss.item()
        test_loss.append(loss.item())

    temp_train_loss, temp_test_loss = train_running_loss/len(train), test_running_loss/len(test)
    print(f'Epoch {epoch}:')
    print(f'Train Loss: {temp_train_loss:.4f} | Epoch Loss: {temp_test_loss:.4f}\n')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 0:
Train Loss: 3.4436 | Epoch Loss: 3.1973



  0%|          | 0/71 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 8:
Train Loss: 2.0769 | Epoch Loss: 1.9296



  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 9:
Train Loss: 2.0265 | Epoch Loss: 1.8820



In [61]:
PATH = '/content/gdrive/MyDrive/Colab Notebooks/Tinkoff DL/Занятие 8/'
fname = 'temp_checkpoint.pt'

In [62]:
torch.save({
    'last_epoch': epoch,
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'train_loss': train_loss,
    'test_loss': test_loss
}, PATH+fname)

## Спеллчекер (3 балла)

Из языковой модели можно сделать простенький спеллчекер: можно визуализировать лоссы на каждом символе (либо какой-нибудь другой показатель неуверенности).

Бонус: можете усреднить перплексии по словам и выделять их, а не отдельные символы.

In [63]:
from IPython.core.display import display, HTML

def print_colored(sequence, intensities, delimeter=''):
    html = delimeter.join([
        f'<span style="background: rgb({255}, {255-x}, {255-x})">{c}</span>'
        for c, x in zip(sequence, intensities) 
    ])
    display(HTML(html))

print_colored('Налейте мне экспрессо'.split(), [0, 0, 100], ' ')

sequence = 'Эту домашку нужно сдать втечении двух недель'
intensities = [0]*len(sequence)
intensities[25] = 50
intensities[26] = 60
intensities[27] = 70
intensities[31] = 150
print_colored(sequence, intensities)

In [70]:
def spellcheck(sequence):
    model.eval()
    loss_func = nn.CrossEntropyLoss(reduction='none')
    tokenized_sequence = torch.LongTensor(dataset.vocab.tokenize(sequence)).reshape(1, -1).to(device)
    # прогоните модель и посчитайте лосс, но не усредняйте
    # с losses можно что-нибудь сделать для визуализации; например, в какую-нибудь степень возвести
    hidden =  model.init_hidden(len(tokenized_sequence)).to(device)
    output, hidden = model(tokenized_sequence, hidden)

    output = output[0]
    tokenized_sequence = tokenized_sequence[0] 
    losses = loss_func(output, tokenized_sequence)
    
    losses = losses ** 2
    # print(losses)
    print_colored(sequence, losses)

In [71]:
sequences = ['В этом претложениии очен много очепяток.', 
             'Здесь появилась лишнняя буква.', 
             'В этом предложении все нормально.', 
             'Чтонибудь пишеться чериз дефис.', 
             'Слова нрпдзх не сущесдвует.']

for sequence in sequences:
    spellcheck(sequence)

Как-то не очень получилось, но это может быть из-за того, что модель недостаточно обучилась, или она очень простая, или я что-то не так сделал.

## Генерация предложений (3 балла)

* Поддерживайте hidden state при генерации. Не пересчитывайте ничего больше одного раза.
* Прикрутите температуру: это когда при сэмплировании все логиты (то, что перед софтмаксом) делятся на какое-то число (по умолчанию 1, тогда ничего не меняется). Температура позволяет делать trade-off между разнообразием и правдоподобием (подробнее — см. блог Карпатого).
* Ваша реализация должна уметь принимать строку seed — то, с чего должно начинаться сгенерированная строка.

In [102]:
def sample(num_tokens, seed="", temperature=1.0):
    model.eval()
    
    input = torch.LongTensor(dataset.vocab.tokenize(seed)).reshape(1, -1).to(device)
    hidden = model.init_hidden(len(input)).to(device)

    continuation = ''
    
    for _ in range(num_tokens):
        output, hidden = model(input, hidden)
        
        token_probas = output.squeeze().div(temperature).exp().cpu()
        token = torch.multinomial(token_probas, 1)[0]

        continuation += dataset.vocab.idx2char[token.item()]# допишите соответствующий символ

        input = seed + continuation
        input = torch.LongTensor(dataset.vocab.tokenize(input)).reshape(1, -1).to(device) # обновите input
        
        # hidden = model.init_hidden(input.shape[1]).to(device)
    
    return continuation

In [107]:
beginnings = ['Шел медведь по лесу', 
              'Встретились англичанин, американец и русский. Англичанин говорит:',
              'Так вот, однажды качки решили делать ремонт',
              'Поручик Ржевский был',
              'Идет Будда с учениками по дороге',
              'Мюллер: Штирлиц, где вы были в 1938 году?',
              'Засылают к нам американцы шпиона под видом студента',
              'Подъезжает электричка к Долгопе:']

for beginning in beginnings:
    print(f'{beginning}... {sample(10, beginning)}')
    print()

Шел медведь по лесу...   авбд  ар

Встретились англичанин, американец и русский. Англичанин говорит:... лгик   иоу

Так вот, однажды качки решили делать ремонт... ья,  ооет 

Поручик Ржевский был... я,ибвиоо и

Идет Будда с учениками по дороге... ,  ев нтбя

Мюллер: Штирлиц, где вы были в 1938 году?... оБиа аатао

Засылают к нам американцы шпиона под видом студента... скииу,,Аич

Подъезжает электричка к Долгопе:... рГ иаериек



Получился настоящий бред (и я совсем не про хлеб), но так как времени до дедлайна почти не осталось, то буду довольствоваться этим.